# pyTorch Convolutional Neural Networks (CNNs)

<div style="padding: 5px; border: 5px solid #a10000ff;">

**Hinweis:** In den Codezellen sind jeweils einige Codeteile nicht programmiert. Diesen Code müssen Sie ergänzen. Die jeweiligen Stellen sind mit einem Kommentar und dem Keyword **TODO** vermerkt und z.T. Stellen mit ... markiert.

Ausserdem gibt es einige assert Statements. Diese geben einen Fehler aus, sollte etwas bei Ihrer Programmierung nicht korrekt sein.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torchvision
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
import os


## Hintergrundinformationen zu CNNs

Convolutional Neural Networks (CNNs) sind eine spezielle Art von künstlichen neuronalen Netzwerken, die besonders gut für die Verarbeitung von Bilddaten geeignet sind. Sie bestehen wie andere neuronale Netze aus mehreren Schichten, darunter Convolutional Layers, Pooling Layers und Fully Connected Layers.

- **Convolutional Layers**: Diese Schichten verwenden Filter (auch als Kernel bezeichnet), um Features im Eingabebild zu erkennen. Dies ermöglicht es dem Netzwerk, Muster wie Kanten, Ecken und Texturen zu erkennen.
- **Pooling Layers**: Diese Schichten reduzieren die räumlichen Dimensionen der Daten, indem sie Informationen zusammenfassen. Dies hilft, die Anzahl der Parameter zu reduzieren und die Rechenleistung zu verbessern.
- **Fully Connected Layers**: Diese Schichten verbinden alle Neuronen der vorherigen Schicht mit allen Neuronen der nächsten Schicht. Sie werden oft am Ende eines CNNs verwendet, um die erkannten Features in eine Klassifikation oder Regression umzuwandeln.

In diesem Notebook werden wir die Grundlagen von CNNs in PyTorch kennenlernen und ein erstes CNN für die Bildklassifikation implementieren.

### Torch vorbereiten

In [ ]:
# Wir prüfen ob eine Hardwarebeschleunigung möglich ist und verwenden diese, wenn sie verfügbar ist
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
torch.manual_seed(42) # Setze den Zufallsseed für Reproduzierbarkeit
print(f"Using {device} device")

## MNIST Dataset vorbereiten, laden und visualisieren

Wir nutzen für dieses Notebook den MNIST-Datensatz, der handgeschriebene Ziffern enthält. Die Ziffern sind in 10 Klassen (0-9) unterteilt, und jede Klasse enthält Tausende von Bildern. 
Die Bilder sind in Graustufen und haben eine Auflösung von 28x28 Pixeln.
Das Modell muss lernen, diese Ziffern korrekt zu klassifizieren.

Wir werden diesen Datensatz laden, vorbereiten und einige Beispiele visualisieren. Der Datensatz kann direkt von PyTorch heruntergeladen werden, was den Prozess vereinfacht.

Quelle Dataset: [MNIST Dataset](https://pytorch.org/vision/stable/datasets.html#mnist)

Originale nicht mehr existierende Webseite: [MNIST Dataset](http://yann.lecun.com/exdb/mnist/)

In [ ]:
# Mnist Dataset laden
mnist_train = torchvision.datasets.MNIST(root='datasets', train=True, download=True)
mnist_test = torchvision.datasets.MNIST(root='datasets', train=False, download=True)

### Machine Learning Prozess
Der Machine Learning Prozess besteht aus mehreren Schritten, die nacheinander durchlaufen werden müssen, um ein funktionierendes Modell zu erstellen. Hier sind die Schritte im Überblick:

<img src="images/ml_process.png" alt="Machine Learning Prozess" width="600"/>

1. **Problemdefinition**: Das Problem, das gelöst werden soll, wird definiert. Es wird festgelegt, welche Art von Modell benötigt wird und welche Daten dafür erforderlich sind.

2. **Daten sammeln& aufbereiten**: Die Daten, die für das Training des Modells benötigt werden, werden gesammelt. Dies kann durch Web-Scraping, APIs, Datenbanken oder andere Quellen erfolgen. Die gesammelten Daten werden bereinigt, transformiert und in ein Format gebracht, das für das Training des Modells geeignet ist. Dies kann das Entfernen von Duplikaten, das Auffüllen/Entfernen von fehlenden Werten oder die Normalisierung der Daten umfassen.

3. **Modell erstellen**: Das Modell wird erstellt, indem die Architektur definiert und die Hyperparameter festgelegt werden. Dies kann die Auswahl von Schichten, Aktivierungsfunktionen, Optimierern und anderen Parametern umfassen.

4. **Modell trainieren & evaluieren**: Das Modell wird mit den vorbereiteten Daten trainiert. Dies umfasst die Auswahl eines Trainingsalgorithmus, die Festlegung der Anzahl der Epochen und die Überwachung des Trainingsfortschritts. Das trainierte Modell wird bewertet, um seine Leistung zu messen. Dies kann die Berechnung von Metriken wie RMSE, Accuracy, Precision oder F1-Score umfassen.

5. **Modell laufen lassen**: Das trainierte und evaluierte Modell wird in einer Produktionsumgebung eingesetzt, um Vorhersagen zu treffen oder Entscheidungen zu unterstützen.

6. **Modell überwachen / ausser Betrieb nehmen**: Das Modell wird kontinuierlich überwacht, um sicherzustellen, dass es weiterhin gute Leistungen erbringt. Dies kann die Überwachung von Metriken, die Erkennung von Datenänderungen oder die Aktualisierung des Modells umfassen. Wenn das Modell nicht mehr benötigt wird oder nicht mehr gut funktioniert, kann es ausser Betrieb genommen werden.

### Daten validieren

Wir prüfen die Daten auf Konsistenz, um sicherzustellen, dass die Daten für das Training eines Modells geeignet sind. In der Praxis wird ein Datensatz auf die Kriterien weiter unten geprüft. Da es ein vorbereiteter Datensatz ist, ist dieser bereits in einem guten Zustand und wir machen nur wenige **Konsistenz-Prüfungen**, um die Datenqualität zu validieren.

**Datenqualitätskriterien:**
- **Vollständigkeit**: Alle notwendigen Daten sind vorhanden, es gibt keine fehlenden Werte.
- **Konsistenz**: Alle Daten sind im gleichen Format (z.B. gleiche Bildgrössen, gleiche Pixelwerte, keine Mischung von Datentypen z.B. Integer und Datum in selbem Feld)
- **Richtigkeit**: Alle Daten sind korrekt und entsprechen der Realität (z.B. keine falschen Labels, keine fehlerhaften Werte)
- **Einzigartigkeit**: Es gibt keine Duplikate in den Daten, die somit die Trainingsdaten verzerren könnten.
- **Aktualität&Relevanz**: Alle Daten sind aktuell und relevant für das Problem, das gelöst werden soll.

Damit wir die Daten benutzen können, müssen alle Daten gleich formiert sein, heisst alle Bilder müssen die gleiche Grösse haben, die Pixelwerte müssen in einem bestimmten Bereich liegen, etc. 
In diesem Abschnitt prüfen wir, ob es verschiedene Bildgrössen gibt, ob alle Pixelwerte von 0 bis 255 reichen und zeigen ein Beispiel eines Bildes an.

In [ ]:
# Die Anzahl Trainings- und Testdaten anzeigen
print(f"Anzahl Trainingsbilder: {len(mnist_train)}")
print(f"Anzahl Testbilder: {len(mnist_test)}")


# Prüfen, dass alle Bilder die gleiche Grösse haben
image_sizes = [image.size for image, _ in mnist_train]
print(f"Einzigartige Bildgrössen: {set(image_sizes)}")

#### Prüfen der Pixelwerte

In [ ]:
# Prüfen, dass die Pixelwerte im Bereich von 0 bis 255 liegen
pixel_values = [image.getdata() for image, _ in mnist_train]
pixel_values_flat = [pixel for sublist in pixel_values for pixel in sublist]
print(f"Min Pixelwert: {min(pixel_values_flat)}")
print(f"Max Pixelwert: {max(pixel_values_flat)}")

# Labels prüfen
labels = [label for _, label in mnist_train]
print(f"Einzigartige Labels: {set(labels)}")

#### Beispiel Data Sample anzeigen

In [ ]:
# Ein Beispielbild aus dem Trainingsdatensatz anzeigen
image, label = mnist_train[0]
print(f"Label: {label}")
plt.imshow(image, cmap='gray')
plt.title(f"Label: {label}")
plt.axis('off')
plt.show()

#### Aufgabe 1

1. Was bedeutet es, wenn die Bilder unterschiedliche Grössen haben? Was könnte das für Probleme verursachen?

> 

2. Wenn wir das Modell auf neue Daten anwenden möchten, was müssen wir sicherstellen, damit das Modell die neuen Daten korrekt verarbeiten kann? Führen Sie dies anhand eines Bildes als Beispiel aus.

> 

### Data Loader erstellen

In diesem Abschnitt erstellen wir wieder einen Data Loader, um die Daten in Batches zu laden und für das Training vorzubereiten. 
Die Daten sind in Trainings- und Testset aufgeteilt.

**Tensorkonvertierung & Normalisierung**: Es wird auch eine Transformation toTensor() von Torchvision angewendet. 
Diese Transformation konvertiert die Bilder in PyTorch-Tensoren, die für das Training des Modells erforderlich sind.
Ausserdem normalisiert sie die Pixelwerte von 0-255 auf einen Bereich von 0-1, was die Stabilität des Trainings verbessert.

In [ ]:
# Damit wir die Bilder in einem CNN verwenden können, müssen wir die Bilder in Tensoren umwandeln.

transform = torchvision.transforms.ToTensor()

# Mit der Bibliothek Torchvision können wird das MNIST Dataset direkt herunterladen und in einem Schritt in Tensoren umwandeln. 
# Train=True lädt den Trainingsdatensatz, Train=False lädt den Testdatensatz.
mnist_transformed_train = torchvision.datasets.MNIST(root='datasets', train=True, download=True, transform=transform)
mnist_transformed_test = torchvision.datasets.MNIST(root='datasets', train=False, download=True, transform=transform)

train_loader = DataLoader(mnist_transformed_train, batch_size=64, shuffle=True)
test_loader = DataLoader(mnist_transformed_test, batch_size=64, shuffle=True)

## Neural Network Architektur definieren

In diesem Abschnitt definieren wir die Architektur unseres Convolutional Neural Networks. 

Wir werden mehrere Convolutional Layers, Pooling Layers und Fully Connected Layers verwenden, um ein Modell zu erstellen, das in der Lage ist, die Bilder im MNIST-Datensatz zu klassifizieren.

### Berechnung der Dimensionen der verschiedenen Layers

In pyTorch müssen wir die Dimensionen der Daten durch die verschiedenen Schichten unseres Netzwerks verfolgen, um die Dimensionen der einzelnen Layers zu konfigurieren. In diesem Abschnitt berechnen wir die Dimensionen der Daten nach jeder Schicht in unserem CNN.

#### Aufgabe 2

Wir möchten folgende Architektur für unser CNN verwenden:
```
Input (28x28) --> Conv1 (7x7, 12 Kernel, Stride 1, Padding 0) --> Pool1 (2x2, Stride 2) --> Conv2 (5x5, 16 Kernel, Stride 1, Padding 2) --> Pool2 (2x2, Stride 2) --> Fully Connected Layer
```




1. Berechnen Sie die Bildgrösse nach dem ersten Convolutional Layer:
Hinweis: Überlegen Sie sich wie sich die Bildgrösse mit dem verwendeten Kernel, Padding und Stride verändert.

>

2. Berechnen Sie die Bildgrösse nach dem ersten Pooling Layer:
> 

3. Berechnen Sie die Bildgrösse nach dem zweiten Convolutional Layer:
> 

4. Berechnen Sie die Bildgrösse nach dem zweiten Pooling Layer:
> 

5. Wie viele Neuronen muss der Fully Connected Layer haben, um die Daten korrekt zu verarbeiten? (Tipp: wir müssen die Anzahl der Kanäle und die Bildgrösse nach dem letzten Pooling Layer berücksichtigen)

> Nach dem zweiten Pooling haben wir 16 Kanäle und die Bildgrösse ist 5x5. Daher müssen wir 16 * 5 * 5 = 400 Neuronen im Fully Connected Layer haben, um alle Informationen aus den vorherigen Schichten zu verarbeiten.


### Aufgabe 3

Programmieren Sie nun die Architektur des CNNs.

**Convolutional Layer**: In PyTorch wird ein Convolutional Layer mit der Funktion `torch.nn.Conv2d` definiert. `torch` steht für PyTorch, `nn` für neural network und `Conv2d` für 2D Convolutional Layer.
```python
torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
```
| Parameter | Erläuterung |
|-----------|-------------|
| `in_channels` | Anzahl der Eingabekanäle. |
| `out_channels` |  Anzahl der Ausgabekanäle (Anzahl Kernels). |
| `kernel_size` | Grösse des Kernels (7x7 Pixel). |
| `stride` | Schrittweite beim Verschieben des Filters. Der Filter wird um 1 Pixel pro Schritt verschoben. |
| `padding` |  Anzahl der Pixel, die um das Eingabebild herum hinzugefügt werden. Mit 0 wird die Bildgrösse reduziert. |


Der **Pooling Layer** wird wie folgt definiert:
```python
torch.nn.MaxPool2d(kernel_size, stride),
```

| Parameter | Erläuterung |
|-----------|-------------|
| `kernel_size` | Grösse des Pooling-Fensters (2x2 Pixel). |
| `stride` | Schrittweite beim Verschieben des "Pooling-Fensters". Der Filter wird um 2 Pixel pro Schritt verschoben. |

Der **Fully Connected Layer** wird mit `nn.Linear` definiert:
```python
nn.Linear(in_features, out_features)
```
| Parameter | Erläuterung |
|-----------|-------------|
| `in_features` | Anzahl der Eingabefeatures (z.B. 400). |
| `out_features` | Anzahl der Ausgabefeatures (z.B. 10 für die 10 Klassen). |


Der **ReLU-Aktivierungsfunktion** wird mit `nn.ReLU()` definiert und fügt Nichtlinearität hinzu, damit das Netzwerk komplexe Muster lernen kann.

Der **Flatten Layer** wird mit `nn.Flatten()` definiert und wandelt die mehrdimensionalen Daten in einen eindimensionalen Vektor um, der für den Fully Connected Layer geeignet ist.

In [ ]:
# TODO erstellen Sie mit torch.nn.sequential ein Convolutional Neural Network mit folgenden Schichten. Die Schichten werden als einzelne Parameter an torch.nn.Sequential übergeben.
# - Convolutional Layer mit einem Eingabe-Kanal, 12 Ausgabekanälen, einem Kernel von 7x7, einem Stride von 1 und einem Padding von 0
# - Max Pooling Layer mit einem Kernel von 2x2 und einem Stride von 2
# - RELU Aktivierungsfunktion
# - Convolutional Layer mit 12 Eingabekanälen, 16 Ausgabekanälen, einem Kernel von 5x5, einem Stride von 1 und einem Padding von 2
# - Max Pooling Layer mit einem Kernel von 2x2 und einem Stride von 2
# - RELU Aktivierungsfunktion
# - Flatten Layer, um die Daten für die Fully Connected Layer vorzubereiten
# - Fully Connected Layer mit der Anzahl Eingabeneuronen die Sie oben berechnet haben und 10 Ausgabeneuronen.

model = torch.nn.Sequential(
    ...
)

## Netzwerk trainieren und evaluieren
In diesem Abschnitt trainieren wir unser CNN mit dem MNIST-Datensatz und evaluieren die Leistung des Modells auf dem Testset. Wir werden die Trainings- und Testgenauigkeit berechnen und die Ergebnisse visualisieren.

### Netzwerk trainieren

Vervollständigen Sie den Trainingsloop, um das CNN zu trainieren.

In [ ]:
# Hyperparameter definieren
max_num_epochs = 10
learning_rate = 0.001
momentum = 0.9

# Optimizer und die Loss-Funktion definieren
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
loss = torch.nn.CrossEntropyLoss()

train_losses = []
test_losses = []

# Das Modell muss noch auf die Hardware verschoben werden.
model.to(device)

for epoch in range(max_num_epochs):
    batch_train_losses = []
    batch_test_losses = []
    for batch in train_loader:
        images, labels = batch

        # Wir verschieben die Bilder und Labels auf die gleiche Hardware wie das Modell (CPU oder GPU)
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        train_loss = loss(outputs, labels)
        train_loss.backward()
        optimizer.step()
        batch_train_losses.append(train_loss.item())

    with torch.no_grad():
        for batch in test_loader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            batch_test_losses.append(loss(outputs, labels).item())

    training_loss = np.mean(batch_train_losses)
    testing_loss = np.mean(batch_test_losses)
    train_losses.append(training_loss)
    test_losses.append(testing_loss)

    print(f"Epoch {epoch}: Train Loss = {training_loss:.5f}, Test Loss = {testing_loss:.5f}")

# Modell speichern
torch.save(model.state_dict(), 'mnist_cnn.pth')


# Test und Trainingsverluste visualisieren
plt.plot(train_losses, label='Trainings Loss')
plt.plot(test_losses, label='Test Loss')
plt.xlabel('Epochen')
plt.ylabel('Loss')
plt.title('Trainings- und Testverlust über Epochen')
plt.legend()
plt.show()


### Modell Laden und Testen
Nachdem wir das Modell trainiert und gespeichert haben, können wir es laden und auf neuen Daten testen. In diesem Abschnitt laden wir das gespeicherte Modell und evaluieren es auf dem Testset, um die Genauigkeit zu berechnen.

In [ ]:
# Testen ob das File für das Modell existiert und das Modell geladen werden kann
if os.path.exists("mnist_cnn.pth"):
    model.load_state_dict(torch.load("mnist_cnn.pth"))
    print("Modell erfolgreich geladen!")


In [ ]:
# Berechne die Accuracy des Modells auf dem Testset
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f"Test Accuracy: {100 * correct / total:.2f}%")

#### Aufgabe 4

Was bedeutet die Accuracy Metrik? Wie gut funktioniert unser Modell?

> 

## Kontrollfragen

1. Wie ist die Architektur eines Convolutional Neural Networks aufgebaut? Welche Schichten werden verwendet und welche Funktionen haben sie?

> 

2. Was ist der Unterschied zwischen einem Convolutional Layer und einem Fully Connected Layer?

> 
3. Wie berechnet man die Eingangsdimensionen des Fully-Connected Layers am Ende des CNNs?

> 